Notebook will be used to create 1 Billion Synthetic data records to be used for various AI Cases.

Total Record: 1 Billion

Customer: > 4.17 Million (Column Name : person_id)

Monthly Expenses : Between 1st day of Month to 28th Day of Month.

Date Range: All months of 2022 and 2023

Importing Libraries:

pandas : to build dataframe (table like structure)

torch: to generate random singular number between 0 and 9

In [2]:
import pandas as pd
import pyarrow
import torch
import random
import time

Creating dataframe and defining the number of records (1 Billion) we want in dataframe

In [3]:
df = pd.DataFrame()
max_df= 100_000_000 # 1 Billion 

Different Expense Category where customer is spending are defined below.

In [4]:
expense_category = {1:'grocery',2:'travel',3:'rent',4:'auto_insurance',5:'hsa',6:'medical',7:'gas',8:'home_furnishing',9:'restaurant',10:'cellphone'}
expense_category_min = {1:10,2:100,3:1500,4:0,5:5,6:0,7:5,8:1,9:4,10:0}
expense_category_max = {1:800,2:5000,3:3000,4:50,5:50,6:50,7:10,8:30,9:20,10:10}


In [5]:
for k,v in expense_category.items():
    df[v] = 0
    print(k,v)

1 grocery
2 travel
3 rent
4 auto_insurance
5 hsa
6 medical
7 gas
8 home_furnishing
9 restaurant
10 cellphone


In [6]:
%%timeit -n 1 -r 1
for k,v in expense_category.items():     
        df[v] = torch.randint(expense_category_min.get(k),expense_category_max.get(k),(max_df+1,),dtype=torch.int16).numpy()

4.82 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000001 entries, 0 to 100000000
Data columns (total 10 columns):
 #   Column           Dtype
---  ------           -----
 0   grocery          int16
 1   travel           int16
 2   rent             int16
 3   auto_insurance   int16
 4   hsa              int16
 5   medical          int16
 6   gas              int16
 7   home_furnishing  int16
 8   restaurant       int16
 9   cellphone        int16
dtypes: int16(10)
memory usage: 1.9 GB


In [16]:
%%timeit -n 1 -r 1
df['day'] = torch.randint(1,29,(df.index[-1]+1,),dtype=torch.int8).numpy()
df['month'] = torch.randint(1,13,(df.index[-1]+1,),dtype=torch.int8).numpy()
df['year'] = torch.randint(2022,2024,(df.index[-1]+1,),dtype=torch.int16).numpy()

855 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
df[['year','month','day']]

,year,month,day
0,2023,1,20
1,2023,11,19
2,2022,11,25
3,2022,3,15
4,2022,2,19
...,...,...,...
99999996,2022,2,28
99999997,2023,6,20
99999998,2022,1,1
99999999,2022,2,19


In [17]:
%%timeit -n 1 -r 1
df['expense_date'] = pd.to_datetime(df[['year','month','day']])

24.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


ymd --> Year Month Day
Below cell is use to achieve how many unique month and year combination we have in synthetic data and it's value counts.

In [21]:
ymd_len = df.loc[df.index>=0,['month','year']].groupby(by=['year','month']).value_counts().values
ymd_list = df.loc[df.index>=0,['month','year']].groupby(by=['year','month']).value_counts().index#[['year','month','day']]


In [22]:
ymd_len.max()

4171757

In [ ]:
df['person_id'] = 0

In [24]:
person_id_start = 10001000 + 10
person_id_end = 10001000 + 10 + ymd_len.max()

In [25]:
for ymd in zip(list(ymd_list),ymd_len):

    val = torch.randint(person_id_start,person_id_end,(len(df.loc[(df.year==ymd[0][0]) & (df.month==ymd[0][1]),].index),)).numpy()
    df.loc[(df.year==ymd[0][0]) & (df.month==ymd[0][1]),'person_id'] = val

Below Cell Output shows the number of Unique Customer created.

In [35]:
f'{df.person_id.nunique()} Unique Number of Customer present in the Synthetic Record Created' 

'4171757 Unique Number of Customer present in the Synthetic Record Created'

Below Cell Command can be used to save the file in parquet format. Parquet is a compressed file format.

In [37]:
df.to_parquet('./100_Million_Customer_Record.pq')

In [38]:
df.loc[df.person_id==10001010,]

,grocery,travel,rent,auto_insurance,hsa,medical,gas,home_furnishing,restaurant,cellphone,day,month,year,expense_date,person_id
3870330,422,1171,2401,49,24,34,5,24,17,5,28,8,2023,2023-08-28,10001010.0
19828908,167,293,2518,44,33,20,7,3,13,6,27,11,2023,2023-11-27,10001010.0
27983124,344,2725,2514,9,35,21,6,18,8,5,17,4,2023,2023-04-17,10001010.0
36501376,121,4300,2108,22,47,11,5,7,6,8,2,3,2023,2023-03-02,10001010.0
40064934,284,2714,2463,26,34,24,8,8,10,1,11,7,2023,2023-07-11,10001010.0
40617555,640,1698,1988,1,23,35,5,6,7,3,27,12,2022,2022-12-27,10001010.0
41459918,154,3869,2207,41,27,2,5,12,19,1,24,3,2022,2022-03-24,10001010.0
49308489,569,3529,1847,21,39,7,8,15,14,3,23,6,2022,2022-06-23,10001010.0
52696256,529,3233,2691,1,5,2,8,25,12,7,9,4,2023,2023-04-09,10001010.0
53978173,696,754,1598,4,13,25,8,29,17,6,20,1,2022,2022-01-20,10001010.0


In [39]:
df.loc[df.expense_date=='2023-06-08',]

,grocery,travel,rent,auto_insurance,hsa,medical,gas,home_furnishing,restaurant,cellphone,day,month,year,expense_date,person_id
1342,642,1714,1687,23,18,45,5,7,5,4,8,6,2023,2023-06-08,14133793.0
2336,615,3964,2307,47,40,32,9,13,12,0,8,6,2023,2023-06-08,11613077.0
4947,714,3806,1590,44,40,18,5,21,11,5,8,6,2023,2023-06-08,10056072.0
7056,701,3619,2675,4,5,48,8,6,17,9,8,6,2023,2023-06-08,12512547.0
7425,472,3726,2674,15,38,47,6,12,5,0,8,6,2023,2023-06-08,13672333.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99998348,138,4479,1520,5,33,24,8,26,10,2,8,6,2023,2023-06-08,13671255.0
99998541,370,4512,2138,5,36,22,7,27,16,3,8,6,2023,2023-06-08,11605573.0
99998987,233,4621,2667,30,44,14,7,15,4,9,8,6,2023,2023-06-08,11228468.0
99999288,171,4216,2621,44,8,12,7,29,15,4,8,6,2023,2023-06-08,10579678.0
